In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> # Khai báo thư viện

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import nltk
import pickle
import tensorflow as tf

from nltk import word_tokenize
from nltk.tokenize import word_tokenize

from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers

from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

> # Bước chuẩn bị

**Đọc lại 2 tập text (train, test) đã tiền xử lý trước đó**

In [ ]:
train_text = pd.read_pickle(r'../input/train-text/train_texts.pkl')
test_text = pd.read_pickle(r'../input/test-text/test_texts.pkl')

len(train_text), len(test_text)

Lấy ra target từ train.csv để dùng trong huấn luyện

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
target = train['target'].values

len(target), target, train.target.value_counts()

**Đồ thị biểu diễn số từ trong mỗi câu ở tập train_text và test_text**

In [ ]:
def get_text_len(texts):
    text_len = []
    for text in texts:
        text_len.append(len(text.split(' ')))
    
    return text_len

def show_barchart(text_len, name):
    text_x = Counter(text_len).keys()
    text_y = Counter(text_len).values()
    
    print(name, len(text_x), text_x)
    print(name, len(text_y), text_y)
    plt.bar(text_x, text_y)
    plt.xlabel('Số từ trong 1 câu') 
    plt.ylabel('Số lượng câu') 
    plt.title('Tập ' + name)
                        
    plt.show()

In [ ]:
train_text_len = get_text_len(train_text)
test_text_len = get_text_len(test_text)

show_barchart(train_text_len, 'train_text')
show_barchart(test_text_len, 'test_text')

Như đã nhận xét ở phần EDA, có thể thấy các câu hỏi thường có độ dài là 1->30 từ, câu dài nhất lần lượt chứa 125 và 120 từ ở tập train, test. Nếu sử dụng độ dài lớn nhất của 1 câu để làm số hàng của ma trận vecto câu thì điều đó là rất bất lợi vì:  
* Số câu có > 30 từ chiếm số lượng rất ít: <1% => không có nhiều ảnh hưởng đến việc huấn luyện mô hình.
* Nếu để số chiều embedding quá lớn thì khi đó một ma trận vecto câu sẽ rất thưa và điều này sẽ gây tốn kém chi phí trong việc huấn luyện.  
==> Vì vậy, em sẽ chỉ lấy độ dài tối đa trong 1 câu là 30 từ.

Chuyển dạng của train_text, test_text từ list -> np.ndarray để cùng dạng với target và để phục vụ tính toán trên ma trận khi áp dụng vào trong model sau này.

In [ ]:
train_text = np.array(train_text)
test_text = np.array(test_text)

Khai báo các tham số quan trọng

In [ ]:
MAX_LEN = 30    # Độ dài của 1 câu 
EMBEDDING_SIZE = 300    # Số chiều embedding khi sử dụng fastText mặc định là (300,) 

> # Embedding

**Load mô hình ngôn ngữ word embedding fastText** (1 trong 3 pre-trained word vector được cho phép sử dụng trong challenge này).   
- fastText là một thư viện để học cách nhúng từ và phân loại văn bản do phòng nghiên cứu AI của Facebook tạo ra. Mô hình cho phép một người tạo ra một thuật toán học không giám sát hoặc học có giám sát để thu được các biểu diễn vectơ cho các từ.
- Nó hỗ trợ cả mô hình CBOW và Skip-Gram. Sự khác biệt chính giữa các mô hình khác và FastText đó là: nó chia từ thành vài cụm với ý nghĩa riêng cho mỗi cụm. 
- Ví dụ: tri-gram của từ 'orange' là: 'org', 'ran', 'ang', 'nge' (bỏ qua ranh giới bắt đầu và kết thúc của từ).  
  * Vectơ embedding cho 'orange' sẽ là tổng của tri-gram này. Các từ hiếm hoặc lỗi chính tả có thể được biểu diễn chính xác vì rất có thể một số n-gam của chúng cũng xuất hiện theo cách khác.
- However, fastText có thể tạo ra các vectơ tốt hơn bằng cách chia từ thành nhiều phần và sử dụng các vectơ cho các phần đó để tạo ra một vectơ cuối cùng cho từ.

In [ ]:
from gensim.models import FastText 

def load_fasttext():
    print('loading word embeddings...')
    embeddings_index = {}
    f = open('../input/wiki-news-300d-1m/wiki-news-300d-1M.vec',encoding='utf-8')
    for line in f:
        values = line.strip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('found %s word vectors' % len(embeddings_index))

    return embeddings_index

In [ ]:
embeddings_index = load_fasttext()

Sau khi load xong mô hình fastText, em **kiểm tra xem tập embeddings_index** (là 1 dictionary form {'word':'word's embedding vector'}) đã **cover được bao nhiêu % số từ có trong các câu text từ 2 tập train và test**

In [ ]:
# Trước tiên, xây dựng 1 vocab chưa tất cả các từ trong các câu text
def build_vocab(texts):
    vocab = {}
    for sentence in texts:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

# Hàm check độ cover với tất các từ
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        if word in embeddings_index:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        else:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))

    return unknown_words

# Lời gọi hàm
def vocab_check_coverage(all_text):
    vocab = build_vocab(all_text)
    print("FastText : ")
    oov_fasttext = check_coverage(vocab, embeddings_index)
    oov_fasttext = {"oov_rate": len(oov_fasttext) / len(vocab), 'oov_words': oov_fasttext}

    return oov_fasttext

In [ ]:
# Xây dựng all_text là list chứa tất cả các câu trong 2 tập train, test
# Thể hiện mức độ cover của thư viện fastText, hay dict embeddings_index, với all_text
all_text = list(train_text) + list(test_text) 
print(len(all_text))
oov_fasttext = vocab_check_coverage(all_text)

Như vậy, fastText đã cover được gần đầy đủ tất cả các từ ở trong vocab cho nên em sẽ không sử dụng thêm các pre-trained word embedding khác để lấp nốt những từ chưa được cover lại nữa.  
Mặc dù em có tham khảo qua các bài đã nộp khác, em thấy nhiều người đã sử dụng kết hợp 2, 3 pre-trained WE bằng cách concat các vecto embedding của các từ ở mỗi thư viện embedding lại với nhau để có vecto của 1 từ dài hơn, hay sử dụng các pre-trained WE để bù lấp các từ còn chưa cover được từ 1 pWE. Tuy nhiên em thấy việc làm như vậy sẽ khiến quá trình huấn luyện trở nên rất lâu mà chỉ cải thiện được kết quả ít.  

**Các bước sử dụng pre-trained word embedding ở 1 mô hình sử dụng thư viện Keras là như sau:** 
1. Chuẩn bị dữ liệu 
   - Tiền xử lý
   - Định dạng các câu text thành các tensor để cho vào mạng nơ-ron. Bước này sẽ sử dụng keras.preprocessing.text.Tokenizer.  
     **Note** em áp dụng trong toàn bộ tất cả các câu text chứ không chỉ áp dụng cho riêng train/test text để tránh trường hợp các từ chưa được gặp trong test text sẽ không có trong train text và điều này sẽ làm mô hình dự đoán kém chính xác đi.
2. Chuẩn bị Embedding Layer
   - Thiết lập các ánh xạ từ 1 từ đến vecto embedding của nó.
   - Có thể sử dụng từ điển embeddings_index và word_index để tính toán ma trận embedding.
3. Xây dựng mô hình với Keras và khai báo Embedding Layer
   - Tại tầng Embedding này, ta sẽ set tham số 'trainable=False' để các vecto embedding không được học trong quá trình huấn luyện mạng.
   
Nguồn: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

(Em đã có tự viết 1 hàm để tự xây dựng lên ma trận embedding cho tất cả các câu. Tuy nhiên, việc xậy dựng thủ công như vậy sẽ chiếm hết bộ nhớ cho phép của kaggle nên em đã chuyển sang sử dụng các ở trên)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_text)

train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

train_padded_sequences = pad_sequences(train_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
test_padded_sequences = pad_sequences(test_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [ ]:
word_index = tokenizer.word_index
num_words = len(word_index)
print('Tổng số từ khác nhau trong all_text: ', num_words)

Trước khi xây dựng lên ma trận embedding cho tất cả các câu, em có muốn áp dụng việc cân bằng dữ liệu với RandomUnderSampling, RandomOverSampling, SMOTE.  
1. Để RandomUnder/RandomOver Sampling thì chỉ cần gọi hàm sau đó các câu text sẽ được ngẫu nhiên loại bỏ/copy để cho dữ liệu với 2 nhãn âm/dương cân bằng. Tuy nhiên, vì dữ liệu chênh lệch là quá lớn cho nên khi Sampling đến khi dữ liệu cân bằng thì ta có thể bị mất/ hoặc sinh thêm quá nhiều dữ liệu và có thể sinh ra nhiều hiện tượng khác như bias, overfitting, ...
2. SMOTE
- Cơ sở: sau bước Chuẩn bị dữ liệu như đã đề cập ở trên, mỗi từ đã có 1 ánh xạ riêng của nó đến tập word_index (như ô code bên dưới), cho nên 1 câu có thể được biểu thành 1 vecto với các giá trị như ở ví dụ bên dưới với train_padded_sequences[0].
- Tuy nhiên, vì số lượng câu có nhãn '0' là rất lớn, hơn nữa, 1 câu được xác định là nhãn '1' có thể nhờ dựa vào 1 vài từ với ngữ cảnh cụ thể mang tính tiêu cực trong câu đó cho còn các từ khác sẽ có tính tích cực. Khi áp dụng SMOTE cho tập minority có thể sẽ sinh ra các ma trận mà giá trị (lấy phần nguyên) sẽ gồm các giá trị của các từ mang tính tích cực nhưng nhãn lại là nhãn '1' (tiêu cực)  
==> Vậy nên, em sẽ không thực hiện bước cân bằng dữ liệu mà em sẽ thay đổi class_weight của tập minority trong quá trình mô hình học với mục đích giúp mô hình dự đoán nhãn thiểu số tốt hơn.

In [ ]:
word_index['name'], word_index['trumph'], train_padded_sequences[0]

**Xây dựng ma trận embedding**

In [ ]:
def prepare_matrix(embedding_index, emb_size = EMBEDDING_SIZE):
    embedding_matrix = np.zeros((num_words+1, emb_size))

    for word, i in word_index.items():
        if i > num_words:
            continue

        emb_vec = embedding_index.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

    return embedding_matrix

In [ ]:
embedding_matrix = prepare_matrix(embeddings_index)
embedding_matrix.shape

Khai báo một số tham số quan trọng trong quá trình training

In [ ]:
LEARNING_RATE = 0.001
BATCH_SIZE = 512
EPOCHS = 10

Vì dữ liệu bị imbalance nên em sẽ dùng độ đo f1 để đánh giá

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

-------------------------------------------------------------------------------------------------------------

## **MODEL**

Một câu text được xác định là có ý tốt hay không thì rất có khả năng là nó sẽ dựa vào ngữ cảnh của cả câu (ngữ cảnh của các từ trong câu) chứ không chỉ đơn thuần phụ thuộc vào một từ hay cụm từ nào đấy.  
Vì vậy, em sử dụng mô hình mạng RNN để ta có thể lưu lại ngắn hạn ý nghĩa của các từ với nhau, cụ thể là từ phía trước tới từ phía sau, để từ đó phần nào lưu lại được ngữ cảnh của cả câu. Thêm nữa,  

In [ ]:
def build_model(embedding_matrix, num_words, EMBEDDING_SIZE):
    inp = Input(shape=(MAX_LEN,))
    x = Embedding(num_words+1, EMBEDDING_SIZE, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.125)(x)
    x1 = Bidirectional(LSTM(256, return_sequences=True))(x)
    x2 = Bidirectional(GRU(256, return_sequences=True))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    predictions = Dense(1, activation='sigmoid')(conc)
    model = Model(inputs=inp, outputs=predictions)
    adam = Adam(lr=LEARNING_RATE)
    model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy',f1])
    return model

In [ ]:
es = EarlyStopping(monitor=f1, mode='max', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_f1', factor=0.1, patience=2, min_=0.0005, verbose=1)

In [ ]:
class_weights = {0: 1, 1: 7}

In [ ]:
def plot_train_val_f1score(hist):
    plt.figure(figsize=(12,8))
    epoch_list = list(range(1,len(hist.history['f1'])+1))
    plt.plot(epoch_list, hist.history['f1'],label='f1')
    plt.plot(epoch_list, hist.history['val_f1'],label='val_f1')
    plt.xlabel('epoches')
    plt.ylabel('score')
    plt.legend()
    plt.show()

def tweak_threshold(pred_prob, truth):
    thresholds = []
    scores = []
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        thresholds.append(thresh)
        score = f1_score(truth, (pred_prob>thresh).astype(int))
        scores.append(score)
    return np.max(scores), thresholds[np.argmax(scores)]

In [ ]:
from sklearn.model_selection import StratifiedKFold
SFold = StratifiedKFold(n_splits=5, random_state=20, shuffle=True)
SFold

In [ ]:
best_score = []
y_test = np.zeros((test_padded_sequences.shape[0], ), dtype=np.float32)
oof_pred = np.zeros((train_padded_sequences.shape[0], ), dtype=np.float32)
val_list = []
hist = []

for tr_index, val_index in SFold.split(train_padded_sequences, target):
    val_list += list(val_index)
    X_tr, X_val, y_tr, y_val = train_padded_sequences[tr_index], train_padded_sequences[val_index], \
                                target[tr_index], target[val_index]
    print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)
    model = build_model(embedding_matrix, num_words, EMBEDDING_SIZE)
    history = model.fit(X_tr, y_tr,
                       batch_size=BATCH_SIZE, 
                       epochs=EPOCHS, 
                       validation_data=(X_val,y_val), 
                       callbacks=[es, reduce_lr],
                       class_weight=class_weights,
                       verbose=1
                       )
    plot_train_val_f1score(history)
    
    oof_pred[val_index] = np.squeeze(model.predict([X_val], batch_size=BATCH_SIZE))
    f1score, threshold = tweak_threshold(oof_pred[val_index], np.squeeze(y_val))
    print(f"F1score: {round(f1score, 4)} for threshold: {threshold} on validation data")
    best_score.append(threshold)
    val_pred = (np.squeeze(oof_pred[val_index]) > threshold).astype(int)
    print('Classification Report on validation data: \n', classification_report(y_val, val_pred))

    y_test += np.squeeze(model.predict([test_padded_sequences], batch_size=BATCH_SIZE))/5
    
    hist.append(history)

**Nhận xét:**  
Sau khi chạy Stratified 5 folds, có thể thấy là:  
- Mô hình đã học được 93-95% dữ liệu, chưa quá tốt.
- Mô hình đã bị overfit    
==> Không tăng số unit trong LSTM và GRU và tăng SpatialDropout lên thành 0.2
- Mô hình hội tụ rất chậm sau 6 hoặc 7 epoch  
==> Không tăng số epoch training lên nữa.
- Điểm precision, recall, f1-score của nhãn '0' là rất cao. Trong khi đó, điểm precision của nhãn '1' lại khá kém, chỉ 52%, điểm recall của nhãn '1' đã khá là tốt. Mặt khác, loss của model giảm ít sau từng epoch, loss vẫn còn khá cao. Điều này có thể là do việc set class_weight cho nhãn '1' của em = 7 là tương đối lớn. Em cũng đã chạy thử model với class_weight cho nhãn '1' là 5 thì model đạt accuracy cao hơn (trên 95%) và loss có thấp hơn 1 chút.  
==> Vì vậy, em sẽ giảm class_weight của nhãn 1 xuống, từ 7->5 và sử dụng Regularization với l2 (l2 sẽ apply penalty on layer parameter nếu weight quá lớn).  
- Việc tăng class_weight của nhãn '1' lên có thể khiến cho việc dự đoán nhãn '0' trở nên không tốt bằng như trước, tuy nhiên, điểm đánh giá của nhãn '0' đang là rất tốt nên ta có thể đánh đổi điểm dự đoán nhãn '0' để tăng được điểm dự đoán ở nhãn '1' vì dù sao nhãn '1' cũng là nhãn quan trọng cần dữ đoán chính xác nhất có thể.

In [ ]:
# In ra threshold tốt nhất cho tập validation test 
f1score, threshold = tweak_threshold(np.squeeze(oof_pred[val_list]), np.squeeze(target[val_list]))
print(f"F1score: {round(f1score, 4)} for threshold: {threshold} on validation data")

-------------------------------------------------------------------------------------------------------------------------------

## **New Model**

In [ ]:
def build_new_model(embedding_matrix, num_words, EMBEDDING_SIZE):
    inp = Input(shape=(MAX_LEN,))
    x = Embedding(num_words+1, EMBEDDING_SIZE, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    weight_decay = regularizers.l2(l2=0.01)
    x1 = Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=weight_decay))(x)
    x2 = Bidirectional(GRU(256, return_sequences=True, kernel_regularizer=weight_decay))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    predictions = Dense(1, activation='sigmoid')(conc)
    model = Model(inputs=inp, outputs=predictions)
    adam = Adam(lr=LEARNING_RATE)
    model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy',f1])
    return model

In [ ]:
new_model = build_new_model(embedding_matrix, num_words, EMBEDDING_SIZE)

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(train_padded_sequences, target, test_size=0.2, shuffle=True, stratify=target)

In [ ]:
new_class_weights = {0: 1, 1: 5}

In [ ]:
new_history = new_model.fit(X_tr, y_tr,
                           batch_size=BATCH_SIZE, 
                           epochs=EPOCHS, 
                           validation_data=(X_val,y_val), 
                           callbacks=[es, reduce_lr],
                           class_weight=new_class_weights,
                           verbose=1
                           )

In [ ]:
plot_train_val_f1score(new_history)

In [ ]:
val_pred = np.squeeze(new_model.predict([X_val], batch_size=BATCH_SIZE))
f1score, threshold = tweak_threshold(val_pred, np.squeeze(y_val))
print(f"F1score: {round(f1score, 4)} for threshold: {threshold} on validation data")
val_prediction = (val_pred > threshold).astype(int)
print('Classification Report on validation data: \n', classification_report(y_val, val_prediction))

Sau khi train với new_model, model bây giờ đã bị underfit và loss vẫn còn lớn. Điều này có thể là do em đã set giá trị của của lambda trong l2 khá lớn.

Em xin tiếp tục ở notebook sau ạ

Thưa thầy, vì 2 lí do:  
1. Do đã chạy nhiều thử nghiệm nên tài khoản kaggle của em đã bị hết thời gian sử dụng GPU để có thể train model nhanh.
2. Vì đây là challenge đã kết thúc nên để nộp được bài thì không được sử dụng nguồn dữ liệu nào khác bên ngoài bằng '+ Add data', mà phần bài phía trên của em là phần notebook riêng để chạy model nên em phải sử dụng dữ liệu đã preprocess từ notebook trước.  
Nên em xin phép được nộp bài có submission ở 1 notebook khác bằng tài khoản em mượn của bạn em ạ.  
Em xin cảm ơn thầy ạ!

--------------------------------------------------------------------------------------------------------------------------

## Sử dụng GridSearch để tìm hyperparameter tốt nhất

Vì thời gian chạy GridSearch là rất lâu và hơn nữa, kaggle không cho em chạy với 

In [ ]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import f1_score

# BS = [256, 512, 1024]
# EP = [15, 20, 25, 30]
# LR = [0.001, 0.01, 0.1, 0.2, 0.3]
# MOMENTUM = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

# model = KerasClassifier(lambda: build_model(embedding_matrix, num_words, EMBEDDING_SIZE))

# param_grid = dict(batch_size=BS, epochs=EP, learn_rate=LR, momentum=MOMENTUM)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1', n_jobs=-1)
# grid_result = grid.fit(x_tr, y_tr)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))